In [59]:
#!pip install ortools

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

In [41]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')
d = time_matrix[:10,:10].tolist()
len(d)

10

In [42]:
name_lst = ['강남역','학동역','현대아파트','신당동.청구역','동묘앞역','안암오거리','서초동삼성아파트','방배역','사당역13번출구','남성초등학교','언북중학교입구','세관앞','신구중학교','금옥초등학교.금호대우아파트','장충체육관앞','대광고등학교앞','서초역2번출구','청호나이스','방배동신동아파트','서울고사거리','방배동래미안아파트','서울교통공사','구름다리','이수역','사당역'][:10]

In [43]:
time_matrix[:10,:10]

array([[   0,  467, 1840,  994, 1420, 1558,  556,  601,  936, 1038],
       [ 636,    0, 1504,  616, 1006, 1221,  986, 1024, 1238, 1221],
       [2241, 1662,    0, 1737, 1797, 1297, 2206, 2464, 2581, 2479],
       [1238,  697, 1828,    0,  390,  737, 1338, 1296, 1510, 1611],
       [1813, 1234, 1456,  938,    0,  346, 1778, 2036, 2153, 2051],
       [1897, 1317, 1216, 1244,  756,    0, 1862, 2119, 2236, 2134],
       [ 506,  820, 1792, 1116, 1644, 1509,    0,  316,  549,  809],
       [ 727, 1091, 1966, 1291, 1818, 1684,  318,    0,  276,  564],
       [ 979, 1210, 2023, 1460, 1886, 1741,  533,  359,    0,  425],
       [ 929, 1055, 1868, 1305, 1731, 1586,  483,  382,  303,    0]])

In [60]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = d
    data['demands'] = [0,3,2] + [1 for i in range(7)]+[0] 
    data['vehicle_capacities'] = [4,4,4]
    data['num_vehicles'] = 3
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0

    grouped_bus_station_lst = []
    

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0
        loop_lst = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            loop_lst.append(node_index)
            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        grouped_bus_station_lst.append(loop_lst)
        plan_output += '서비스 종료\n\n'
        plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('\n')
    print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
    print('Total load of all routes: {}'.format(total_load))
   
    grouped_bus_station = pd.DataFrame({'Vehicle':[i for i in range(3)],'bus_station':grouped_bus_station_lst})
    
    with open("grouped_bus_station.pickle", "wb") as f:
	    pickle.dump(grouped_bus_station, f)

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        8000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)

    
  
if __name__ == '__main__':
    main()

Objective: 518983
Route for vehicle 0:

 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 방배역  (탑승객 수: 3, 하차수: 1, 소요시간: 0:10:01) 
    ==>     정류장 이름 : 사당역13번출구  (탑승객 수: 2, 하차수: 1, 소요시간: 0:04:36) 
    ==>     정류장 이름 : 남성초등학교  (탑승객 수: 1, 하차수: 1, 소요시간: 0:07:05) 
    ==>     정류장 이름 : 서초동삼성아파트  (탑승객 수: 0, 하차수: 1, 소요시간: 0:08:03) 
    ==>    서비스 종료

Duration of the route: 0:29:45분
Load of the route: 4

Route for vehicle 1:

 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 안암오거리  (탑승객 수: 3, 하차수: 1, 소요시간: 0:25:58) 
    ==>     정류장 이름 : 학동역  (탑승객 수: 0, 하차수: 3, 소요시간: 0:21:57) 
    ==>    서비스 종료

Duration of the route: 0:47:55분
Load of the route: 4

Route for vehicle 2:

 정류장 이름 : 강남역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 신당동.청구역  (탑승객 수: 3, 하차수: 1, 소요시간: 0:16:34) 
    ==>     정류장 이름 : 동묘앞역  (탑승객 수: 2, 하차수: 1, 소요시간: 0:06:30) 
    ==>     정류장 이름 : 현대아파트  (탑승객 수: 0, 하차수: 2, 소요시간: 0:24:16) 
    ==>    서비스 종료

Duration of the route: 0:47:20